In [ ]:
import QuantLib as ql

In [ ]:
import csv
import os


def get_project_root():
    '''
    Returns the project root.
    '''
    return os.getcwd()


def read_swap_rates_from_file(file_name):
    root_dir = get_project_root()
    file_path = os.path.join(root_dir, 'data/', file_name)
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        rates = [(ql.PeriodParser.parse(str(r[0])), float(r[1])) for r in reader]
    return rates

In [ ]:
def calculate_last_liquid_forward(crv: ql.YieldTermStructureHandle, fsp: ql.Period):
    dc = crv.dayCounter()
    omega = 8.0 / 15.0
    fsp_date = crv.referenceDate() + fsp
    cut_off = crv.timeFromReference(fsp_date)
    llfr_weights = ((ql.Period(25, ql.Years), 1.0), 
                    (ql.Period(30, ql.Years), 0.5), 
                    (ql.Period(40, ql.Years), 0.25), 
                    (ql.Period(50, ql.Years), 0.125))
    llfr = 0.0
    for tenor, weight in llfr_weights:
        time_to_maturity = crv.timeFromReference(fsp_date + tenor)
        llfr += weight * crv.forwardRate(
            cut_off, time_to_maturity, ql.Continuous, ql.NoFrequency, True).rate()
    return llfr


In [ ]:
def bump_quote(quote_handle: ql.RelinkableQuoteHandle, bump = 0.0001):
    quote_handle.linkTo(ql.SimpleQuote(quote_handle.value() + bump))

In [ ]:
# SWAP INDEX CONVENTIONS

ftk_crv_handle = ql.RelinkableYieldTermStructureHandle()

settlement_days = 2
business_convention = ql.Unadjusted
day_count = ql.SimpleDayCounter()
calendar = ql.NullCalendar()
ccy = ql.EURCurrency()
fixed_frequency = ql.Annual
floating_tenor = ql.Period(6, ql.Months)

ftk_idx = ql.IborIndex("FTK_IDX", floating_tenor, settlement_days, ccy, calendar,
                       business_convention, False, day_count, ftk_crv_handle)

In [ ]:
# UFR CONVENTIONS

ufr_compounded = ql.InterestRate(0.023, day_count, ql.Compounded, ql.Annual)
ufr_continuous = ufr_compounded.equivalentRate(ql.Continuous, ql.Annual, 1.0).rate()
ufr_quote_handle = ql.QuoteHandle(ql.SimpleQuote(ufr_continuous))

first_smoothing_point = ql.Period(20, ql.Years)
alpha = 0.1;

In [ ]:
llfr_quote_handle = ql.RelinkableQuoteHandle()

def update_llfr_handle():
    llfr = calculate_last_liquid_forward(ftk_crv_handle, first_smoothing_point)
    llfr_quote_handle.linkTo(ql.SimpleQuote(llfr))
    print('LLFR observer notified.')
    

observer = ql.Observer(update_llfr_handle)

In [ ]:
# VALUATION DATE

today = calendar.adjust(ql.Date(29, ql.March, 2019))
ql.Settings.instance().evaluationDate = today
settlement = calendar.advance(today, settlement_days, ql.Days)

In [ ]:
# SWAP RATES

# swap_rates = [(ql.Period(1, ql.Years), -0.00315), 
#               (ql.Period(2, ql.Years), -0.00205), 
#               (ql.Period(3, ql.Years), -0.00144),
#               (ql.Period(4, ql.Years), -0.00068),
#               (ql.Period(5, ql.Years), 0.00014),  
#               (ql.Period(6, ql.Years), 0.00103),
#               (ql.Period(7, ql.Years), 0.00194),  
#               (ql.Period(8, ql.Years), 0.00288),  
#               (ql.Period(9, ql.Years), 0.00381),
#               (ql.Period(10, ql.Years), 0.00471), 
#               (ql.Period(12, ql.Years), 0.0063),  
#               (ql.Period(15, ql.Years), 0.00808),
#               (ql.Period(20, ql.Years), 0.00973), 
#               (ql.Period(25, ql.Years), 0.01035), 
#               (ql.Period(30, ql.Years), 0.01055),
#               (ql.Period(40, ql.Years), 0.0103), 
#               (ql.Period(50, ql.Years), 0.0103)]
swap_rates = read_swap_rates_from_file('swap_rates_29032019.csv')

swap_rate_quotes = [(q[0], ql.RelinkableQuoteHandle(ql.SimpleQuote(q[1]))) for q in swap_rates]

# Link swap quotes to observer

for _, quote_handle in swap_rate_quotes:
    observer.registerWith(quote_handle)

In [ ]:
# FTK CURVE CONSTRUCTION
      
instruments = [ql.SwapRateHelper(q, t, calendar, fixed_frequency, business_convention, day_count, ftk_idx) 
               for t, q in swap_rate_quotes]
ftk_curve = ql.PiecewiseLogLinearDiscount(settlement, instruments, day_count)
ftk_curve.enableExtrapolation()
            
ftk_crv_handle.linkTo(ftk_curve)

In [ ]:
# Link swap quotes to observer

for _, quote_handle in swap_rate_quotes:
    observer.registerWith(quote_handle)

# UFR CURVE CONSTRUCTION
update_llfr_handle()
ufr_curve = ql.UltimateForwardTermStructure(ftk_crv_handle, llfr_quote_handle, ufr_quote_handle, first_smoothing_point, alpha)

In [ ]:
# print(swap_rate_quotes[0][1].value())
# bump_quote(swap_rate_quotes[15][1], 0.001)
# print(swap_rate_quotes[0][1].value())

# ftk_crv_handle.discount(ftk_crv_handle.referenceDate() + ql.Period(50, ql.Years))
ufr_curve.discount(ftk_crv_handle.referenceDate() + ql.Period(50, ql.Years))

In [ ]:
0.42445379975356323

In [ ]:
llfr_quote_handle.value()

In [ ]:
0.02003708049348038

In [ ]:
ql.PeriodParser.parse('1y')